# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
# CONFIG = or_suite.envs.env_configs.resource_allocation_foodbank_config(6)
CONFIG = or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 10

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
  'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
  'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
#  'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[2.059 1.983 2.018 2.02  1.952 2.026 2.042 1.929 1.999 2.022]
 [2.961 2.966 3.016 2.939 3.04  3.014 2.972 3.032 3.01  3.023]
 [4.086 3.991 3.955 4.154 4.064 4.077 4.052 4.009 4.01  4.082]] [[1.01858677 1.0122801  0.99180442 0.98772466 0.9918145  1.04943985
  1.01795678 0.94231576 1.02029359 1.02348229]
 [1.44273317 1.42788095 1.38482634 1.38465844 1.40370937 1.35418019
  1.38463569 1.40462664 1.43104857 1.38725304]
 [1.77217493 1.66760877 1.73406315 1.80229964 1.70818734 1.82238059
  1.73818756 1.73058343 1.75781114 1.7467902 ]]
Mean and variance endomwnets:
[[2.039 2.021 2.01  1.957 1.983 2.006 2.016 2.018 2.022 2.028]
 [2.938 3.033 2.857 3.025 3.008 2.992 3.053 2.976 3.05  2.977]
 [4.052 4.076 3.93  3.976 4.004 3.936 4.086 3.987 4.09  3.927]] [[1.02346422 1.03274343 0.99493718 0.94082464 1.01819006 0.9715781
  1.03910731 1.01176875 1.01070075 1.0115414 ]
 [1.37264562 1.41488904 1.3994824  1.42631518 1.45737298 1.40567991
  1.4649884  1.38687563 1.401962

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = { 'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG),
                    'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG),
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG),
#                     'Budget': lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)
                    }
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

FixedThreshold
Lower and Upper Solutions:
[[0.      0.14378]
 [0.      0.14378]
 [0.18275 0.     ]]
Lower and Upper Solutions:
[[0.      0.13817]
 [0.      0.13817]
 [0.17287 0.     ]]
Lower and Upper Solutions:
[[0.      0.13618]
 [0.      0.13618]
 [0.1839  0.     ]]
Lower and Upper Solutions:
[[ 0.       0.14239]
 [-0.       0.14239]
 [ 0.16769  0.     ]]
Lower and Upper Solutions:
[[0.      0.13356]
 [0.      0.13356]
 [0.17038 0.     ]]
Lower and Upper Solutions:
[[0.      0.1389 ]
 [0.      0.1389 ]
 [0.17466 0.     ]]
Lower and Upper Solutions:
[[0.      0.13817]
 [0.      0.13817]
 [0.17721 0.     ]]
Lower and Upper Solutions:
[[0.      0.14239]
 [0.      0.14239]
 [0.1719  0.     ]]
Lower and Upper Solutions:
[[0.      0.13502]
 [0.      0.13502]
 [0.18591 0.     ]]
Lower and Upper Solutions:
[[0.      0.13951]
 [0.      0.13951]
 [0.18471 0.     ]]
Lower and Upper Solutions:
[[0.      0.13403]
 [0.      0.13403]
 [0.17633 0.     ]]
Lower and Upper Solutions:
[[0.      0.13678

Lower and Upper Solutions:
[[0.      0.1431 ]
 [0.      0.1431 ]
 [0.17178 0.     ]]
[[0.      0.28688]
 [0.      0.28688]
 [0.34438 0.     ]]
Lower and Upper Solutions:
[[0.      0.13738]
 [0.      0.13738]
 [0.18082 0.     ]]
[[ 0.       0.27541]
 [-0.       0.27541]
 [ 0.3625   0.     ]]
Lower and Upper Solutions:
[[0.      0.13943]
 [0.      0.13943]
 [0.16218 0.     ]]
[[0.      0.27952]
 [0.      0.27952]
 [0.32514 0.     ]]
Lower and Upper Solutions:
[[ 0.       0.13682]
 [-0.       0.13682]
 [ 0.17024  0.     ]]
[[0.      0.27429]
 [0.      0.27429]
 [0.3413  0.     ]]
Lower and Upper Solutions:
[[0.      0.13955]
 [0.      0.13955]
 [0.1745  0.     ]]
[[ 0.       0.27977]
 [-0.       0.27977]
 [ 0.34983  0.     ]]
Lower and Upper Solutions:
[[0.      0.13614]
 [0.      0.13614]
 [0.17272 0.     ]]
[[0.      0.27293]
 [0.      0.27293]
 [0.34626 0.     ]]
Writing to file data.csv
        Algorithm    Reward      Time     Space  Efficiency  Hindsight Envy  \
0  FixedThreshold -9

In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)